<b>Loading Libraries</b>

In [53]:
import time
import pprint
import glob
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
import string
import pickle
import gensim
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import stem_text
from nltk.tokenize import word_tokenize

<b>Preprocessing and tokenizing function</b>

In [54]:
def preprocess(text):
    return [word for word in word_tokenize(stem_text(strip_punctuation(text.lower().replace("\n", "")))) if word not in ['lrb', 'rrb']]

<b>Initializing global variables </b>

In [55]:
pp = pprint.PrettyPrinter(indent=2)
files = glob.glob('/home/rohan/Documents/Homework/WSTA/wiki-pages-text/*.txt')
files = [file for file in files if file != '/home/rohan/Documents/Homework/WSTA/wiki-pages-text/to_test.txt']
files = ['/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-001.txt']
doc_dict = {}
file_dict = {}
df = pd.DataFrame()
stemmer = nltk.stem.PorterStemmer()
punc_table = str.maketrans({key: None for key in string.punctuation})

<b> Reading files into a dictionary </b>

In [56]:
start = time.time()
for file in files:
    with open(file, encoding = 'utf-8') as f:
        print(file)
        for line in f:
            try:
                #print(line)
                line = line.split(maxsplit=2)
                #print(line)
                key = line[0]
                value = line[2]
                if key in file_dict.keys():
                    file_dict[key] = file_dict[key] + value
                else:
                    file_dict[key] = value
                    #print(value)
            except Exception as e:
                print("ERROR",e, line)
        for key, values in file_dict.items():
            doc_dict[key] = values
        #print(file_dict)
print("Time taken: ", time.time() - start)

/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-001.txt
Time taken:  0.20307421684265137


<b> Loading into Dataframe </b>

In [57]:
df = pd.DataFrame(doc_dict.items(),columns=['doc_id','doc'])

<b> Pickle dataframe </b>

In [115]:
pkl_name = file.split('/')[-1].split(".")[0]
df.to_pickle("all_files" + ".pickle")

<b> Load pickled dataframe </b>

In [46]:
df = pd.read_pickle('all_files.pickle')

<b> Print dataframe and preprocess document</b>

In [58]:
#print(df.head())
#print(df.head()['value'].tolist())
start = time.time()
df['doc_processed'] = df['doc'].apply(lambda x: preprocess(x))
print("Number of documents:", len(df))
print("Time taken:", time.time() - start)

Number of documents: 48228
Time taken: 20.75375771522522


<b> Pickling preprocessed docs </b>

NameError: name 'punc_less_docs' is not defined

<b> Create Gensim dictionary </b>

In [19]:
start = time.time()
dictionary = gensim.corpora.Dictionary(df['doc_processed'].tolist())
print(dictionary)
print("Time taken:", time.time() - start)

Dictionary(60085 unique tokens: ['1928', 'ar', 'event', 'follow', 'footbal']...)
Time taken: 1.6684720516204834


<b> Creating BOW represention of documents </b>

In [20]:
start = time.time()
corpus = [dictionary.doc2bow(doc) for doc in df['value'].tolist()]
print(corpus[:5])
print("Time taken:", time.time() - start)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 4), (8, 1), (9, 1), (10, 1)], [(3, 1), (5, 4), (7, 32), (9, 1), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 1), (21, 1), (22, 1), (23, 2), (24, 6), (25, 1), (26, 2), (27, 3), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 4), (40, 2), (41, 4), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 11), (52, 1), (53, 1), (54, 1), (55, 1), (56, 5), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 4), (64, 3), (65, 3), (66, 1), (67, 4), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 12), (81, 1), (82, 1), (83, 1), (84, 3), (85, 1), (86, 1), (87, 1), (88, 4), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 4), (96, 1), (97, 1), (98, 1), (99, 4), (100, 3), (101, 2), (102, 1), (103, 2), (104, 1), (105, 1), (1

<b> Creating tf-idf model from this corpus </b>

In [21]:
start = time.time()
tf_idf = gensim.models.TfidfModel(corpus)
s = 0
for i in corpus:
    s += len(i)
print(s)
print("Time taken:", time.time() - start)

2017570
Time taken: 0.39360857009887695


<b> Creating Similarity index </b>

In [22]:
start = time.time()
sims = gensim.similarities.Similarity('/home/rohan/Documents/Homework/WSTA',tf_idf[corpus], num_features=len(dictionary))
print("Time taken: ", time.time() - start)

Time taken:  15.522789716720581


<b> Getting query and showing documents related to the query </b>

In [23]:
query = input("Enter statement: ")
start = time.time()
query_doc = preprocess(query)
query_doc_bow = dictionary.doc2bow(query_doc)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)
sim_list = sims[query_doc_tf_idf]
df['similarity_score'] = sim_list
print(type(sim_list), len(sim_list))
print(df.nlargest(5, ['similarity_score']))
#print("Position of most relevant doc: ", max_index)
print("Time taken:", time.time() - start)

Enter statement:  Nicholas scored the winning goal in second half


[(7, 0.0038322623904329446), (63, 0.018533273875172247), (123, 0.20557314586856415), (200, 0.2946698961248762), (303, 0.18485970486134062), (374, 0.33995338406718256), (608, 0.41660702931168775), (1425, 0.7397623502502912)]
<class 'numpy.ndarray'> 48228
                                                     key  \
71                        1987_Football_League_Cup_Final   
18951               1981_American_League_Division_Series   
14385                     1982_Scottish_League_Cup_Final   
44488                      1976–77_Iraqi_National_League   
8426   1914_All-Ireland_Senior_Football_Championship_...   

                                                   value  similarity_score  
71     [the, 1987, footbal, leagu, cup, final, wa, a,...          0.348573  
18951  [the, 1981, american, leagu, divis, seri, ald,...          0.276400  
14385  [the, 1982, scottish, leagu, cup, final, wa, p...          0.271995  
44488  [the, 1976, 77, iraqi, nation, leagu, wa, the,...          0.240028  
